In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# Isochrones

Recall that stellar clusters are gravitationally bound groupings of stars born at the same time.  By looking at the H-R diagram of a cluster -- specifically where the population _leaves_ the main sequence -- we can determine the age of the cluster.  To do this we must know the typical main sequence lifetime of stars, which depends on their mass, metalicity, etc. and for that we must turn to stellar modeling.  [MESA](https://docs.mesastar.org) is a stellar modeling library which incorporates the physics important for stellar evolution into 1-D models of stars.  The results useful for our purposes are stellar evolution tracks, the expected trajectory of the star through the H-R diagram as it evolves in time.

[MIST](http://waps.cfa.harvard.edu/MIST/index.html) is an effort to supply a large database of MESA stellar track simulations for the purpose of producing isochrones.  If you imagine stacking up a cluster's worth of stellar evolution tracks, slicing through them at fixed age will produce an isochrone.

I've used the [MIST isochrone interpolator](http://waps.cfa.harvard.edu/MIST/interp_isos.html) to produce several isochrones based on known properties of M4 (metalicity, reddening due to dust, etc.), which I've included in the `data/` directory.

There is a lot of info in these files.  To save ourselves a _big_ headache in parsing them, we'll make use of a python script authored by [Jieun Choi](https://github.com/jieunchoi).

In [ ]:
!wget -nc https://github.com/jieunchoi/MIST_codes/raw/master/scripts/read_mist_models.py

Now let's read in the file.

In [ ]:
import read_mist_models

filename = '../data/m4_isochrones.iso.cmd'
iso = read_mist_models.ISOCMD(filename)

The information we're after is in `iso.isocmds` (CMD: color-magnitude diagram).

In [ ]:
type(iso.isocmds), len(iso.isocmds)

We see this is a list with 7 elements.  These 7 CMDs are isochrones for 7 different ages, which we can see in the table of each.  Let's inspect the first one.

In [ ]:
iso_array = iso.isocmds[0]
type(iso_array)

This are numpy record arrays (basically numpy arrays with names columns).

In [ ]:
iso_array.dtype.names

For a more detailed description see [here](http://waps.cfa.harvard.edu/MIST/README_tables.pdf).  Most of these columns are synthetic photometry (magnitude measurements) for various telescopes and filters (we'll be using the `Gaia_X_EDR3` ones).  `log10_isochrone_age_yr` is what it sounds like, and is different for each of the 7 CMDs in the list.

In [ ]:
# Compute our color quantity (BP - RP) for the isochrone and plot G vs BP-RP for the isochrone. Don't forget to invert the y-axis!

This includes _all_ phases of stellar evolution.  If our focus is to date the cluster, we really only care about the main sequence and evolutionary phases immediately after (red giant) to find the turnoff.  `phase` indicates this the stellar evolution phase along the isochrone.  We can select the phases we're after:

In [ ]:
phase_sel = (iso_array['phase'] >= 0) & (iso_array['phase'] < 3)

In [ ]:
# Use the selection array to downselect the isochrone and plot it

One final tweak we must make is to account for the distance to the cluster.  Our y-axis isn't actually luminosity, it's the _apparent_ magnitude in the G-band.  What we've been calling `mg` in our previous use of Gaia data was actually derived from the G-band observations and corrected for the distance to each sourse based on parallax measurement (which gave us something like an intrinsic, rather than apparent, brightness).  We're going to know use the true G-band measurement, which is `phot_g_mean_mag`.

The isochrone simulation doesn't account for the faintness we would expect, so we'll need to do it ourselves by applying a [distance modulus](https://en.wikipedia.org/wiki/Distance_modulus).

The cluster is 2.2 kpc away.  We'll use some convenience functions from astropy to compute the corresponding distance modulus.

In [ ]:
!pip install --quiet astropy

In [ ]:
import astropy.coordinates as coord
import astropy.units as u

distance = 2.2 * u.kpc
distmod = coord.Distance(distance).distmod.value
distmod

In [ ]:
# Add this distance modulus to the G-band magnitude of the isochrone

# Overplot Gaia Observation

Finally, let's overplot the Gaia observations of confidently identified members of the M4 cluster.  Read in the cluster catalog we've worked with previously (`NGC6121-1.dat`), Gaia objects in the M4-neigborhood (`m4_gaia_source.csv`), and cross-match the catalogs to find the Gaia observations corresponding to the identified cluster members.

In [ ]:
# Read M4 (NGC6121) and Gaia catalogd and crossmatch them

Now plot the M4 cluster members with the isochrone!

In [ ]:
# plot M4 members and isochrone

How do they compare?  Explore the other isochrones.  Based on the fits (comparing by eye is sufficient), how old do you think 